In [16]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
import librosa
import numpy as np

# Define file paths
positive_folder = os.path.join('./Audio/drone')
negative_folder = os.path.join('./Audio/not_drone')

# Load files into TensorFlow datasets
pos = tf.data.Dataset.list_files(positive_folder + '/*.wav')
neg = tf.data.Dataset.list_files(negative_folder + '/*.wav')

# Assign labels to the datasets
positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))))

# Combine the datasets
data = positives.concatenate(negatives)

# Function to load and preprocess audio files
def load_wav_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    return wav, sample_rate

# Function to convert audio to spectrogram
def audio_to_spectrogram(wav, sample_rate):
    spectrogram = librosa.feature.melspectrogram(y=wav.numpy(), sr=sample_rate.numpy(), n_mels=128)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    return spectrogram

# Function to process each element in the dataset
def preprocess(file_path, label):
    wav, sample_rate = load_wav_mono(file_path)
    spectrogram = tf.py_function(audio_to_spectrogram, [wav, sample_rate], tf.float32)
    return spectrogram, label

# Apply the preprocessing function to the dataset
data = data.map(lambda file_path, label: preprocess(file_path, label))

# Batch the data
data = data.batch(32)

# Prefetch the data for performance optimization
data = data.prefetch(tf.data.experimental.AUTOTUNE)

# Print the dataset to verify
for spectrogram, label in data.take(1):
    print(spectrogram.shape, label)
    
    
    

(32, 128, 862) tf.Tensor(
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0.], shape=(32,), dtype=float32)


2024-10-19 17:23:53.501376: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [17]:

# Calculate the length of each audio file
lengths = []
for file in os.listdir(positive_folder):
    tensor_wave = load_wav_mono(os.path.join(positive_folder, file))
    lengths.append(len(tensor_wave))

# Convert lengths to a TensorFlow tensor
lengths_tensor = tf.constant(lengths, dtype=tf.int32)

# Calculate mean, min, and max lengths
mean_length = tf.math.reduce_mean(lengths_tensor)
min_length = tf.math.reduce_min(lengths_tensor)
max_length = tf.math.reduce_max(lengths_tensor)

# Print the results
print(f"Mean length: {mean_length.numpy()}")
print(f"Min length: {min_length.numpy()}")
print(f"Max length: {max_length.numpy()}")




Mean length: 2
Min length: 2
Max length: 2
